In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score

import statsmodels.api as sm

In [ ]:
total_data = pd.read_csv('/content/drive/MyDrive/final_project/data/final_dataset/total_data_v5.csv', encoding='cp949')

<ipython-input-17-e899b3b8de5f>:1: DtypeWarning: Columns (10,11,19,20,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv('/content/drive/MyDrive/final_project/data/final_dataset/total_data_v5.csv', encoding='cp949')


In [ ]:
total_data = total_data.sort_values(by=['아파트명','법정동주소'])

total_data['전분기 가격'] = total_data.groupby('아파트명')['단위면적당거래금액'].shift(1)

total_data['전분기 대비 가격변동률'] = (total_data['단위면적당거래금액'] - total_data['전분기 가격']) / total_data['전분기 가격'] * 100

In [ ]:
df = total_data[total_data['분기']>'2017 2Q']
df['temp'] = df['도로명주소'] + '_' + df['전용면적구간'].astype(str)
group = df.groupby(['temp'])['결측치연속'].agg(['max']).reset_index()
to_del = group[group['max']>5]['temp']
df = df[~df['temp'].isin(to_del)]
df.drop('temp', axis=1, inplace=True)
df.shape

<ipython-input-19-9e7606eef42e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp'] = df['도로명주소'] + '_' + df['전용면적구간'].astype(str)


(590600, 62)

In [ ]:
df = df[~((df['전분기 가격'] == 0.0) & (df['전분기 대비 가격변동률'] == np.inf))]

In [ ]:
df.info()

In [ ]:
df['도'] = df['도로명주소'].apply(lambda x: x.split()[0])
df['시군'] = df['도로명주소'].apply(lambda x: ' '+x.split()[1] if x.split()[1].endswith(('시','군','구')) else '')
df['구'] = df['도로명주소'].apply(lambda x: ' '+x.split()[2] if x.split()[2].endswith('구') else '')
df['지역'] = df['도'] + df['시군'] + df['구']

In [ ]:
df[(df['투기지역'] == True)]['분기'].unique()

array(['2017 3Q', '2017 4Q', '2018 1Q', '2018 2Q', '2018 3Q', '2018 4Q',
       '2019 1Q', '2019 2Q', '2019 3Q', '2019 4Q', '2020 1Q', '2020 2Q',
       '2020 3Q', '2020 4Q', '2021 1Q', '2021 2Q', '2021 3Q', '2021 4Q',
       '2022 1Q', '2022 2Q', '2022 3Q', '2022 4Q', '2023 1Q', '2023 2Q',
       '2023 3Q'], dtype=object)

In [ ]:
df['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

# 강력 권고 모델링 기준 분기: ‘17.3Q / ‘19.3Q / ‘21.3Q / ‘22.4Q / ‘23,3Q

# 2017 3Q

In [ ]:
selected_quarters2 = [['서울시'], ['경기도'], ['인천시'], ['서울시', '경기도', '인천시'], ['부산시'], ['대구시'], ['광주시'], ['울산시'],
                      ['대전시'], ['세종시'], ['경상남도'], ['경상북도'], ['전라남도'], ['전라북도'], ['충청남도'], ['충청북도'],
                      ['경상남도', '경상북도'], ['전라남도', '전라북도'], ['충청남도', '충청북도'], ['강원도'], ['제주도']]

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

In [57]:
selected_quarters = ['2017 3Q']
selected_quarters2 = ['경기도']

df_2017_3Q = df[df['분기'].isin(selected_quarters)]
df_2017_3Q = df_2017_3Q[df_2017_3Q['도'].isin(selected_quarters2)]

selected_columns = [ '세대수', '용적률', '건폐율', '현관구조', '건설사',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']

# 방수 욕실수 전용률 전분기 대비 가격변동률 현관구조

df_subset_2017_3Q = df_2017_3Q[selected_columns]

scaler = MinMaxScaler()

# '분기' 컬럼을 제외한 컬럼 리스트
columns_to_normalize = ["용적률", "건폐율", "중학교_학업성취도", "중학교_최단거리", "건축년도"]

# 정규화 진행
df_subset_2017_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2017_3Q[columns_to_normalize])

X = df_subset_2017_3Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2017_3Q['단위면적당거래금액']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

evs = explained_variance_score(y_test, y_pred)
print(f'Explained Variance Score: {evs}')

<ipython-input-57-1a8d72d9057b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset_2017_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2017_3Q[columns_to_normalize])


Mean Squared Error: 9857.398847139548
R-squared: 0.861478924325904
Mean Absolute Error (MAE): 62.013136278802
Root Mean Squared Error (RMSE): 99.28443406264422
Explained Variance Score: 0.861554039546771


In [58]:
feature_importances = model.feature_importances_

indices = np.argsort(feature_importances)[::-1]

feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

sorted_feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(sorted_feature_importance_df)

Feature Importances:
         Feature  Importance
15        직주_근접도    0.257193
11     중학교_학업성취도    0.162064
9      간접역세권_노선수    0.145299
17          건축년도    0.113427
18     백화점쇼핑몰_유무    0.044772
0            세대수    0.039613
1            용적률    0.036737
6       세대당_주차대수    0.032329
7         전용면적구간    0.030590
12      중학교_최단거리    0.025883
16       학원가_근접도    0.022730
2            건폐율    0.017406
14      스타벅스_매장수    0.016895
5         임대세대비율    0.014892
10  고속철_직접역세권_여부    0.011248
8      직접역세권_노선수    0.008341
19         마트_유무    0.007132
13   초등학교_최단거리구간    0.005960
4            건설사    0.005237
3           현관구조    0.002251


In [59]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     550.4
Date:                Tue, 09 Jan 2024   Prob (F-statistic):               0.00
Time:                        09:14:27   Log-Likelihood:                -53360.
No. Observations:                8093   AIC:                         1.068e+05
Df Residuals:                    8072   BIC:                         1.069e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           20.7569     14.205      1.461   

In [ ]:
df['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

# 2019 3Q

In [ ]:
selected_quarters2 = [['서울시'], ['인천시'], ['경기도'], ['서울시', '경기도', '인천시'], ['부산시'], ['대구시'], ['광주시'], ['울산시'],
                      ['대전시'], ['세종시'], ['경상남도'], ['경상북도'], ['전라남도'], ['전라북도'], ['충청남도'], ['충청북도'],
                      ['경상남도', '경상북도'], ['전라남도', '전라북도'], ['충청남도', '충청북도'], ['강원도'], ['제주도']]

In [60]:
selected_quarters = ['2019 3Q']
selected_quarters2 = ['인천시', '서울시', '경기도']

df_2019_3Q = df[df['분기'].isin(selected_quarters)]
df_2019_3Q = df_2019_3Q[df_2019_3Q['도'].isin(selected_quarters2)]

selected_columns = [ '세대수', '용적률', '건폐율', '현관구조', '건설사',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']

# '조정대상지구', '투기지역', '투기과열지구' '전분기 대비 가격변동률'
df_subset_2019_3Q = df_2019_3Q[selected_columns]

scaler = MinMaxScaler()

# '분기' 컬럼을 제외한 컬럼 리스트
columns_to_normalize = ["용적률", "건폐율", "중학교_학업성취도", "중학교_최단거리", "건축년도"]

# 정규화 진행
df_subset_2019_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2019_3Q[columns_to_normalize])

X = df_subset_2019_3Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2019_3Q['단위면적당거래금액']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

evs = explained_variance_score(y_test, y_pred)
print(f'Explained Variance Score: {evs}')

<ipython-input-60-dc68fc6086c5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset_2019_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2019_3Q[columns_to_normalize])


Mean Squared Error: 18001.000965643292
R-squared: 0.8911026226226723
Mean Absolute Error (MAE): 83.04162942714007
Root Mean Squared Error (RMSE): 134.16780897683054
Explained Variance Score: 0.8911506726617149


In [61]:
feature_importances = model.feature_importances_

indices = np.argsort(feature_importances)[::-1]

feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

sorted_feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(sorted_feature_importance_df)

Feature Importances:
         Feature  Importance
15        직주_근접도    0.278549
9      간접역세권_노선수    0.159789
11     중학교_학업성취도    0.141338
17          건축년도    0.100535
18     백화점쇼핑몰_유무    0.039512
0            세대수    0.037027
1            용적률    0.036674
6       세대당_주차대수    0.031876
12      중학교_최단거리    0.028204
7         전용면적구간    0.024919
5         임대세대비율    0.020682
2            건폐율    0.019353
16       학원가_근접도    0.019208
14      스타벅스_매장수    0.018904
10  고속철_직접역세권_여부    0.013445
8      직접역세권_노선수    0.008964
19         마트_유무    0.007316
4            건설사    0.006284
13   초등학교_최단거리구간    0.005245
3           현관구조    0.002177


In [62]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.578
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                     557.1
Date:                Tue, 09 Jan 2024   Prob (F-statistic):               0.00
Time:                        09:15:13   Log-Likelihood:                -56929.
No. Observations:                8148   AIC:                         1.139e+05
Df Residuals:                    8127   BIC:                         1.140e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -58.7885     21.004     -2.799   

In [ ]:
df_2019_3Q['도'].unique()

array(['제주도'], dtype=object)

In [ ]:
df_2019_3Q['분기'].unique()

array(['2019 3Q'], dtype=object)

In [ ]:
df['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

# 21.3Q 상승장

In [ ]:
selected_quarters2 = [['서울시'], ['인천시'], ['경기도'], ['서울시', '경기도', '인천시'], ['부산시'], ['대구시'], ['광주시'], ['울산시'],
                      ['대전시'], ['세종시'], ['경상남도'], ['경상북도'], ['전라남도'], ['전라북도'], ['충청남도'], ['충청북도'],
                      ['경상남도', '경상북도'], ['전라남도', '전라북도'], ['충청남도', '충청북도'], ['강원도'], ['제주도']]

In [63]:
selected_quarters = ['2021 3Q']
selected_quarters2 = ['인천시', '서울시', '경기도']

df_2021_3Q = df[df['분기'].isin(selected_quarters)]
df_2021_3Q = df_2021_3Q[df_2021_3Q['도'].isin(selected_quarters2)]

selected_columns = [ '세대수', '용적률', '건폐율', '현관구조', '건설사',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']

# 투기지역 투기과열지구 조정대상지구 현관구조
# 방수 욕실수 전용율
df_subset_2021_3Q = df_2021_3Q[selected_columns]

scaler = MinMaxScaler()

# '분기' 컬럼을 제외한 컬럼 리스트
columns_to_normalize = ["용적률", "건폐율", "중학교_학업성취도", "중학교_최단거리", "건축년도"]

# 정규화 진행
df_subset_2021_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2021_3Q[columns_to_normalize])

X = df_subset_2021_3Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2021_3Q['단위면적당거래금액']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

evs = explained_variance_score(y_test, y_pred)
print(f'Explained Variance Score: {evs}')

<ipython-input-63-b52dad7a29cb>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset_2021_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2021_3Q[columns_to_normalize])


Mean Squared Error: 43517.57135749396
R-squared: 0.8715523141330781
Mean Absolute Error (MAE): 136.24462827312723
Root Mean Squared Error (RMSE): 208.6086559984843
Explained Variance Score: 0.8715835763297408


In [64]:
# 단위면적당거래금액
feature_importances = model.feature_importances_

sorted_indices = np.argsort(feature_importances)[::-1]

sorted_feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns[sorted_indices],
    'Importance': feature_importances[sorted_indices]
})

print("Feature Importances:")
print(sorted_feature_importance_df)

Feature Importances:
         Feature  Importance
0         직주_근접도    0.284058
1      중학교_학업성취도    0.137292
2      간접역세권_노선수    0.130435
3           건축년도    0.085945
4            세대수    0.054680
5      백화점쇼핑몰_유무    0.051331
6            용적률    0.039393
7       세대당_주차대수    0.035425
8         전용면적구간    0.033083
9       중학교_최단거리    0.029180
10       학원가_근접도    0.028905
11           건폐율    0.020978
12      스타벅스_매장수    0.015611
13        임대세대비율    0.014176
14  고속철_직접역세권_여부    0.011018
15         마트_유무    0.007934
16     직접역세권_노선수    0.007496
17           건설사    0.005338
18   초등학교_최단거리구간    0.005322
19          현관구조    0.002401


In [65]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     637.0
Date:                Tue, 09 Jan 2024   Prob (F-statistic):               0.00
Time:                        09:15:49   Log-Likelihood:                -59420.
No. Observations:                8148   AIC:                         1.189e+05
Df Residuals:                    8127   BIC:                         1.190e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -60.9693     28.514     -2.138   

In [ ]:
df_2021_3Q['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

In [ ]:
df_2021_3Q['분기'].unique()

array(['2021 3Q'], dtype=object)

In [ ]:
correlations = df_subset_2021_3Q.corr()
unit_price_correlations = correlations['단위면적당거래금액'].sort_values(ascending = False)
print(unit_price_correlations)

단위면적당거래금액       1.000000
건축년도            0.468575
세대당_주차대수        0.448505
세대수             0.397302
중학교_학업성취도       0.291126
건설사             0.212998
백화점쇼핑몰_유무       0.202460
간접역세권_노선수       0.202157
스타벅스_매장수        0.188332
마트_유무           0.183329
전용면적구간          0.181497
고속철_직접역세권_여부    0.126324
직접역세권_노선수       0.075785
직주_근접도          0.054389
용적률             0.039918
임대세대비율         -0.022659
건폐율            -0.026627
중학교_최단거리       -0.063744
현관구조           -0.176159
초등학교_최단거리구간    -0.185990
학원가_근접도              NaN
Name: 단위면적당거래금액, dtype: float64


In [ ]:
df['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

# 22.4Q 하락장

In [ ]:
selected_quarters2 = [['서울시'], ['인천시'], ['경기도'], ['서울시', '경기도', '인천시'], ['부산시'], ['대구시'], ['광주시'], ['울산시'],
                      ['대전시'], ['세종시'], ['경상남도'], ['경상북도'], ['전라남도'], ['전라북도'], ['충청남도'], ['충청북도'],
                      ['경상남도', '경상북도'], ['전라남도', '전라북도'], ['충청남도', '충청북도'], ['강원도'], ['제주도']]

In [66]:
selected_quarters = ['2022 4Q']
selected_quarters2 = ['인천시', '서울시', '경기도']

df_2022_4Q = df[df['분기'].isin(selected_quarters)]
df_2022_4Q = df_2022_4Q[df_2022_4Q['도'].isin(selected_quarters2)]

selected_columns = [ '세대수', '용적률', '건폐율', '현관구조', '건설사',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']

# 방수 욕실수 전용률 투기과열지구 조정대상지구 투기지역

df_subset_2022_4Q = df_2022_4Q[selected_columns]

scaler = MinMaxScaler()

# '분기' 컬럼을 제외한 컬럼 리스트
columns_to_normalize = ["용적률", "건폐율", "중학교_학업성취도", "중학교_최단거리", "건축년도"]

# 정규화 진행
df_subset_2022_4Q[columns_to_normalize] = scaler.fit_transform(df_subset_2022_4Q[columns_to_normalize])

X = df_subset_2022_4Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2022_4Q['단위면적당거래금액']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

evs = explained_variance_score(y_test, y_pred)
print(f'Explained Variance Score: {evs}')

<ipython-input-66-fc426a507cd9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset_2022_4Q[columns_to_normalize] = scaler.fit_transform(df_subset_2022_4Q[columns_to_normalize])


Mean Squared Error: 51349.07617719158
R-squared: 0.8441618083463668
Mean Absolute Error (MAE): 149.4428865470533
Root Mean Squared Error (RMSE): 226.60334546778338
Explained Variance Score: 0.8444219789431534


In [67]:
# 단위면적당거래금액
feature_importances = model.feature_importances_

sorted_indices = np.argsort(feature_importances)[::-1]

sorted_feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns[sorted_indices],
    'Importance': feature_importances[sorted_indices]
})

print("Feature Importances:")
print(sorted_feature_importance_df)

Feature Importances:
         Feature  Importance
0         직주_근접도    0.276380
1      간접역세권_노선수    0.145913
2      중학교_학업성취도    0.141972
3           건축년도    0.082872
4      백화점쇼핑몰_유무    0.042138
5       세대당_주차대수    0.041713
6            용적률    0.041482
7            세대수    0.033774
8       중학교_최단거리    0.033225
9        학원가_근접도    0.029045
10        전용면적구간    0.027652
11           건폐율    0.022434
12        임대세대비율    0.020329
13      스타벅스_매장수    0.015461
14  고속철_직접역세권_여부    0.013599
15     직접역세권_노선수    0.008201
16         마트_유무    0.008069
17   초등학교_최단거리구간    0.006941
18           건설사    0.005770
19          현관구조    0.003033


In [68]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     505.6
Date:                Tue, 09 Jan 2024   Prob (F-statistic):               0.00
Time:                        09:16:32   Log-Likelihood:                -59993.
No. Observations:                8148   AIC:                         1.200e+05
Df Residuals:                    8127   BIC:                         1.202e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           67.7377     30.589      2.214   

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

model = LinearRegression()

# 모델 훈련
model.fit(X_train, y_train)

# 회귀 계수 출력
coefficients = model.coef_
intercept = model.intercept_

# 결과 출력
print("회귀 계수 (Coefficients):")
for i, coef in enumerate(coefficients):
    print(f"독립 변수 {i+1}: {coef}")

print(f"상수항 (Intercept): {intercept}")

회귀 계수 (Coefficients):
독립 변수 1: 0.04561967440062023
독립 변수 2: -685.5885509366057
독립 변수 3: -422.08266523030204
독립 변수 4: -17.218224689716887
독립 변수 5: 147.8063669047835
독립 변수 6: 350.685028434091
독립 변수 7: 144.59826607711554
독립 변수 8: -64.10424490506774
독립 변수 9: 49.580370377002495
독립 변수 10: 53.20501436069125
독립 변수 11: 135.18270317074666
독립 변수 12: 940.6673188851132
독립 변수 13: 96.35776219951437
독립 변수 14: 34.32209980131726
독립 변수 15: 45.116013875219686
독립 변수 16: 243.31146901472977
독립 변수 17: 274.4945160725972
독립 변수 18: 5.8812090797288334
독립 변수 19: -529.8989743439895
독립 변수 20: -54.23032060269316
상수항 (Intercept): 841.3987203944033


In [ ]:
df_2022_4Q['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

In [ ]:
df_2022_4Q['분기'].unique()

array(['2022 4Q'], dtype=object)

# 23,3Q 현재분기

In [ ]:
selected_quarters2 = [['서울시'], ['인천시'], ['경기도'], ['서울시', '경기도', '인천시'], ['부산시'], ['대구시'], ['광주시'], ['울산시'],
                      ['대전시'], ['세종시'], ['경상남도'], ['경상북도'], ['전라남도'], ['전라북도'], ['충청남도'], ['충청북도'],
                      ['경상남도', '경상북도'], ['전라남도', '전라북도'], ['충청남도', '충청북도'], ['강원도'], ['제주도']]

In [69]:
selected_quarters = ['2023 3Q']
selected_quarters2 = ['인천시', '서울시', '경기도']

df_2023_3Q = df[df['분기'].isin(selected_quarters)]
df_2023_3Q = df_2023_3Q[df_2023_3Q['도'].isin(selected_quarters2)]

selected_columns = [ '세대수', '용적률', '건폐율', '현관구조', '건설사',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']

# 방수 욕실수 전용률 투기과열지구 조정대상지구 투기지역

df_subset_2023_3Q = df_2023_3Q[selected_columns]

scaler = MinMaxScaler()

# '분기' 컬럼을 제외한 컬럼 리스트
columns_to_normalize = ["용적률", "건폐율", "중학교_학업성취도", "중학교_최단거리", "건축년도"]

# 정규화 진행
df_subset_2023_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2023_3Q[columns_to_normalize])

X = df_subset_2023_3Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2023_3Q['단위면적당거래금액']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

evs = explained_variance_score(y_test, y_pred)
print(f'Explained Variance Score: {evs}')

<ipython-input-69-7712ff7095a7>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset_2023_3Q[columns_to_normalize] = scaler.fit_transform(df_subset_2023_3Q[columns_to_normalize])


Mean Squared Error: 34119.0103927166
R-squared: 0.8796213445700326
Mean Absolute Error (MAE): 116.51266970371549
Root Mean Squared Error (RMSE): 184.71331947836518
Explained Variance Score: 0.8797457616946895


In [70]:
# 단위면적당거래금액
feature_importances = model.feature_importances_

sorted_indices = np.argsort(feature_importances)[::-1]

sorted_feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns[sorted_indices],
    'Importance': feature_importances[sorted_indices]
})

print("Feature Importances:")
print(sorted_feature_importance_df)

Feature Importances:
         Feature  Importance
0         직주_근접도    0.278456
1      중학교_학업성취도    0.154383
2      간접역세권_노선수    0.143105
3           건축년도    0.098069
4            용적률    0.039889
5      백화점쇼핑몰_유무    0.039554
6            세대수    0.036246
7       세대당_주차대수    0.036156
8       중학교_최단거리    0.026573
9        학원가_근접도    0.025044
10        전용면적구간    0.020901
11           건폐율    0.020858
12      스타벅스_매장수    0.017853
13        임대세대비율    0.016368
14  고속철_직접역세권_여부    0.015809
15         마트_유무    0.008935
16     직접역세권_노선수    0.008541
17   초등학교_최단거리구간    0.006489
18           건설사    0.004607
19          현관구조    0.002166


In [71]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.575
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     550.5
Date:                Tue, 09 Jan 2024   Prob (F-statistic):               0.00
Time:                        09:16:48   Log-Likelihood:                -59280.
No. Observations:                8148   AIC:                         1.186e+05
Df Residuals:                    8127   BIC:                         1.187e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -25.8287     28.028     -0.922   

In [ ]:
df_2023_3Q['도'].unique()

array(['전라북도'], dtype=object)

In [ ]:
df_2023_3Q['분기'].unique()

array(['2023 3Q'], dtype=object)

# 통계치 검정

In [ ]:
X_train = X_train.astype(float)

In [ ]:
y_train = y_train.astype(float)

In [ ]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     1218.
Date:                Wed, 03 Jan 2024   Prob (F-statistic):               0.00
Time:                        07:56:24   Log-Likelihood:            -1.1804e+05
No. Observations:               18649   AIC:                         2.361e+05
Df Residuals:                   18624   BIC:                         2.363e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -114.3518     14.399     -7.941   

In [ ]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                 3.001e+04
Date:                Wed, 03 Jan 2024   Prob (F-statistic):               0.00
Time:                        07:14:08   Log-Likelihood:            -3.8923e+06
No. Observations:              559488   AIC:                         7.785e+06
Df Residuals:                  559463   BIC:                         7.785e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -345.8991      2.891   -119.631   

* 전분기 가격을 빼니 모든 변수가 0.05이하의 유의미함을 나타냄

In [ ]:
df_vif = df[[ '세대수', '용적률', '건폐율', '현관구조', '건설사', '방수', '욕실수',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']]

In [ ]:
df_vif = df_vif.astype(int)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
'''
df_vif["조정대상지구"] = df_vif["조정대상지구"].astype(int)
df_vif["투기지역"] = df_vif["투기지역"].astype(int)
df_vif["투기과열지구"] = df_vif["투기과열지구"].astype(int)
'''
# VIF 계산 함수
def calculate_vif(df_vif):
    vif = pd.DataFrame()
    vif["variables"] = df_vif.columns
    vif["VIF"] = [variance_inflation_factor(df_vif.values, i) for i in range(df_vif.shape[1])]
    return vif

# VIF 계산
vif_df = calculate_vif(df_vif)

sorted_vif_df = vif_df.sort_values(by="VIF", ascending=False)
sorted_vif_df
# 방수 전용율 욕실수 중학교_최단거리

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,variables,VIF
19,건축년도,144.185649
13,중학교_학업성취도,80.789645
5,방수,63.326650
6,욕실수,28.625377
1,용적률,7.328157
9,전용면적구간,6.514522
21,마트_유무,5.088072
2,건폐율,5.062038
22,단위면적당거래금액,4.492895
20,백화점쇼핑몰_유무,4.327434


In [ ]:
correlation_matrix = df_vif.corr()['건축년도'].sort_values(ascending=False)
print(correlation_matrix)

건축년도            1.000000
세대당_주차대수        0.573296
욕실수             0.524148
용적률             0.288704
방수              0.230128
전용면적구간          0.220259
건폐율             0.191043
건설사             0.179453
단위면적당거래금액       0.142114
세대수             0.131355
중학교_최단거리        0.095300
중학교_학업성취도       0.052622
초등학교_최단거리구간     0.050408
직주_근접도          0.046607
스타벅스_매장수        0.027286
직접역세권_노선수      -0.039772
학원가_근접도        -0.042342
간접역세권_노선수      -0.047048
고속철_직접역세권_여부   -0.050815
마트_유무          -0.072464
백화점쇼핑몰_유무      -0.086603
현관구조           -0.266391
임대세대비율               NaN
Name: 건축년도, dtype: float64


In [ ]:
data = {
    "Feature": ["건축년도", "세대당_주차대수", "욕실수", "용적률", "방수", "전용면적구간", "건폐율", "건설사", "단위면적당거래금액", "세대수",
                "중학교_최단거리", "중학교_학업성취도", "초등학교_최단거리구간", "직주_근접도", "스타벅스_매장수", "직접역세권_노선수", "학원가_근접도",
                "간접역세권_노선수", "고속철_직접역세권_여부", "마트_유무", "백화점쇼핑몰_유무", "현관구조"],
    "Correlation": [1.000000, 0.573296, 0.524148, 0.288704, 0.230128, 0.220259, 0.191043, 0.179453, 0.142114, 0.131355,
                    0.095300, 0.052622, 0.050408, 0.046607, 0.027286, -0.039772, -0.042342, -0.047048, -0.050815,
                    -0.072464, -0.086603, -0.266391]
}

# 데이터 프레임 생성
df = pd.DataFrame(data)
df

,Feature,Correlation
0,건축년도,1.000000
1,세대당_주차대수,0.573296
2,욕실수,0.524148
3,용적률,0.288704
4,방수,0.230128
5,전용면적구간,0.220259
6,건폐율,0.191043
7,건설사,0.179453
8,단위면적당거래금액,0.142114
9,세대수,0.131355


In [ ]:
correlation_matrix = df_vif.corr()['중학교_학업성취도'].sort_values(ascending=False)
print(correlation_matrix)

중학교_학업성취도       1.000000
단위면적당거래금액       0.316696
학원가_근접도         0.311628
백화점쇼핑몰_유무       0.282969
스타벅스_매장수        0.211530
세대수             0.184824
직주_근접도          0.179181
간접역세권_노선수       0.175605
세대당_주차대수        0.175295
마트_유무           0.172258
전용면적구간          0.159028
건설사             0.130960
직접역세권_노선수       0.128865
방수              0.112548
욕실수             0.111617
용적률             0.060984
건축년도            0.052622
고속철_직접역세권_여부   -0.004714
건폐율            -0.014421
현관구조           -0.033771
중학교_최단거리       -0.141735
초등학교_최단거리구간    -0.171842
임대세대비율               NaN
Name: 중학교_학업성취도, dtype: float64


In [ ]:
data = {
    "Feature": ["중학교_학업성취도", "단위면적당거래금액", "학원가_근접도", "백화점쇼핑몰_유무", "스타벅스_매장수", "세대수", "직주_근접도",
                "간접역세권_노선수", "세대당_주차대수", "마트_유무", "전용면적구간", "건설사", "직접역세권_노선수", "방수", "욕실수",
                "용적률", "건축년도", "고속철_직접역세권_여부", "건폐율", "현관구조", "중학교_최단거리", "초등학교_최단거리구간", "임대세대비율"],
    "Correlation": [1.000000, 0.316696, 0.311628, 0.282969, 0.211530, 0.184824, 0.179181, 0.175605, 0.175295, 0.172258,
                    0.159028, 0.130960, 0.128865, 0.112548, 0.111617, 0.060984, 0.052622, -0.004714, -0.014421, -0.033771,
                    -0.141735, -0.171842, None]
}

# 데이터 프레임 생성
df = pd.DataFrame(data)
df

,Feature,Correlation
0,중학교_학업성취도,1.000000
1,단위면적당거래금액,0.316696
2,학원가_근접도,0.311628
3,백화점쇼핑몰_유무,0.282969
4,스타벅스_매장수,0.211530
5,세대수,0.184824
6,직주_근접도,0.179181
7,간접역세권_노선수,0.175605
8,세대당_주차대수,0.175295
9,마트_유무,0.172258


In [ ]:
correlation_matrix = df.corr()['전용율'].sort_values(ascending=False)
print(correlation_matrix)

<ipython-input-163-f61858aecb58>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()['전용율'].sort_values(ascending=False)


전용율             1.000000
전용면적구간          0.367048
방수              0.256245
욕실수             0.090119
longitude       0.076479
결측치연속           0.075489
학원가_최단거리        0.057106
직주_최단거리         0.047939
마트_유무           0.038608
백화점쇼핑몰_최단거리     0.023763
고속철_직접역세권_여부    0.023395
평균거래금액          0.008590
전분기 대비 가격변동률    0.007764
중학교_학업성취도       0.001634
투기지역           -0.000135
학원가_근접도        -0.000610
백화점쇼핑몰_유무      -0.003150
조정대상지구         -0.008292
투기과열지구         -0.008292
초등학교_최단거리구간    -0.009305
초등학교_최단거리      -0.014256
직접역세권_노선수      -0.014391
스타벅스_매장수       -0.016879
건설사            -0.022073
직주_근접도         -0.024173
초등학교_설립정보      -0.024729
간접역세권_노선수      -0.034683
마트_최단거리        -0.039060
중학교_졸업자수       -0.039151
중학교_최단거리       -0.043236
전용면적합          -0.051256
임대세대비율         -0.054044
거래금액합          -0.069799
latitude       -0.076635
거래존재여부         -0.082472
초등학교_학생수       -0.092751
세대수            -0.104344
건폐율            -0.117725
거래량            -0.136306
단위면적당거래금액      -0.137484


In [ ]:
y_train_log_transformed = np.log(y_train)

In [ ]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.OLS(y_train_log_transformed, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              단위면적당거래금액   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                 1.048e+05
Date:                Tue, 02 Jan 2024   Prob (F-statistic):               0.00
Time:                        04:50:41   Log-Likelihood:                 20108.
No. Observations:              372992   AIC:                        -4.016e+04
Df Residuals:                  372965   BIC:                        -3.987e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.5345      0.005    825.212   

In [ ]:
X_train_sm = sm.add_constant(X_train)  # 상수항 추가
model = sm.GLM(y_train_log_transformed, X_train_sm).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              단위면적당거래금액   No. Observations:               372992
Model:                            GLM   Df Residuals:                   372965
Model Family:                Gaussian   Df Model:                           26
Link Function:               Identity   Scale:                        0.052569
Method:                          IRLS   Log-Likelihood:                 20108.
Date:                Tue, 02 Jan 2024   Deviance:                       19607.
Time:                        04:58:23   Pearson chi2:                 1.96e+04
No. Iterations:                     3   Pseudo R-squ. (CS):             0.9993
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.5345      0.005    825.212   

* 경기도, 서울, 대구, 인천, 대전
* 경남, 강원도
* 2022 1분기

# 22년 1분기

In [ ]:
df['도'].unique()

array(['경상남도', '경상북도', '대구시', '서울시', '부산시', '경기도', '광주시', '인천시', '충청남도',
       '강원도', '전라남도', '전라북도', '울산시', '대전시', '세종시', '충청북도', '제주도'],
      dtype=object)

In [ ]:
selected_quarters = ['2022 1Q']
selected_quarters2 = ['경상남도', '강원도']

df_2022_1Q = df[df['분기'].isin(selected_quarters)]
df_2022_1Q = df_2022_1Q[df_2022_1Q['도'].isin(selected_quarters2)]

In [ ]:
df_2022_1Q['도'].unique()

array(['경상남도', '강원도'], dtype=object)

In [ ]:
selected_columns = [ '세대수', '용적률', '건폐율', '현관구조', '건설사',
                    '임대세대비율', '세대당_주차대수', '전용면적구간', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부',
                    '중학교_학업성취도', '중학교_최단거리', '초등학교_최단거리구간', '스타벅스_매장수', '직주_근접도', '학원가_근접도',
                    '건축년도', '백화점쇼핑몰_유무', '마트_유무', '단위면적당거래금액']

# 투기지역 투기과열지구 조정대상지구
# 방수 욕실수 전용율
df_subset_2022_1Q = df_2022_1Q[selected_columns]

In [ ]:
scaler = MinMaxScaler()

# '분기' 컬럼을 제외한 컬럼 리스트
columns_to_normalize = ["용적률", "건폐율", "중학교_학업성취도", "중학교_최단거리", "건축년도"]

# 정규화 진행
df_subset_2022_1Q[columns_to_normalize] = scaler.fit_transform(df_subset_2022_1Q[columns_to_normalize])

<ipython-input-67-73d81d764a74>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset_2022_1Q[columns_to_normalize] = scaler.fit_transform(df_subset_2022_1Q[columns_to_normalize])


In [ ]:
X = df_subset_2022_1Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2022_1Q['단위면적당거래금액']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

evs = explained_variance_score(y_test, y_pred)
print(f'Explained Variance Score: {evs}')

Mean Squared Error: 2763.888567875521
R-squared: 0.8464810689557207
Mean Absolute Error (MAE): 37.477176873688656
Root Mean Squared Error (RMSE): 52.57269793224921
Explained Variance Score: 0.8466852177708288


In [ ]:
# 단위면적당거래금액
feature_importances = model.feature_importances_

sorted_indices = np.argsort(feature_importances)[::-1]

sorted_feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns[sorted_indices],
    'Importance': feature_importances[sorted_indices]
})

print("Feature Importances:")
print(sorted_feature_importance_df)

Feature Importances:
         Feature  Importance
0           건축년도    0.304124
1       세대당_주차대수    0.236194
2            세대수    0.129681
3      중학교_학업성취도    0.128794
4       중학교_최단거리    0.043910
5            건폐율    0.033736
6            용적률    0.032149
7   고속철_직접역세권_여부    0.018077
8          마트_유무    0.014222
9         전용면적구간    0.013608
10   초등학교_최단거리구간    0.009950
11      스타벅스_매장수    0.009416
12     간접역세권_노선수    0.006505
13     백화점쇼핑몰_유무    0.005886
14           건설사    0.005549
15        직주_근접도    0.002925
16        임대세대비율    0.002433
17          현관구조    0.001958
18     직접역세권_노선수    0.000882
19       학원가_근접도    0.000000


In [ ]:
df_subset_2022_1Q.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2635 entries, 822082 to 1133973
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   세대수           2635 non-null   int64  
 1   용적률           2635 non-null   int64  
 2   건폐율           2635 non-null   int64  
 3   현관구조          2635 non-null   int64  
 4   건설사           2635 non-null   int64  
 5   임대세대비율        2635 non-null   float64
 6   세대당_주차대수      2635 non-null   float64
 7   전용면적구간        2635 non-null   int64  
 8   직접역세권_노선수     2635 non-null   int64  
 9   간접역세권_노선수     2635 non-null   int64  
 10  고속철_직접역세권_여부  2635 non-null   int64  
 11  중학교_학업성취도     2635 non-null   float64
 12  중학교_최단거리      2635 non-null   float64
 13  초등학교_최단거리구간   2635 non-null   int64  
 14  스타벅스_매장수      2635 non-null   int64  
 15  직주_근접도        2635 non-null   int64  
 16  학원가_근접도       2635 non-null   int64  
 17  건축년도          2635 non-null   float64
 18  백화점쇼핑몰_유무     2635 n

In [ ]:
import pandas as pd

# Data for the two different quarters
data_2022_q4_decline = {
    "Feature": ["투기지역", "중학교_학업성취도", "투기과열지구", "건축년도", "직주_근접도", "세대수", "전용면적구간", "건폐율", "용적률",
                "세대당_주차대수", "간접역세권_노선수", "중학교_최단거리", "백화점쇼핑몰_유무", "학원가_근접도", "스타벅스_매장수",
                "고속철_직접역세권_여부", "건설사", "조정대상지구", "직접역세권_노선수", "초등학교_최단거리구간", "마트_유무", "임대세대비율", "현관구조"],
    "Importance": [0.412253, 0.119165, 0.084196, 0.058413, 0.049739, 0.043258, 0.029525, 0.026559, 0.026538,
                   0.026326, 0.023879, 0.020073, 0.017192, 0.014192, 0.010315, 0.007858, 0.007689, 0.006034,
                   0.004851, 0.003609, 0.003280, 0.002921, 0.002135]
}

data_2022_q4_rise = {
    "Feature": ["건축년도", "조정대상지구", "세대당_주차대수", "세대수", "중학교_학업성취도", "용적률", "중학교_최단거리", "건폐율",
                "마트_유무", "전용면적구간", "스타벅스_매장수", "초등학교_최단거리구간", "고속철_직접역세권_여부", "간접역세권_노선수",
                "임대세대비율", "건설사", "현관구조", "백화점쇼핑몰_유무", "투기지역", "직주_근접도", "직접역세권_노선수", "투기과열지구", "학원가_근접도"],
    "Importance": [0.254643, 0.218733, 0.180958, 0.097604, 0.056460, 0.045121, 0.035790, 0.026044, 0.016533,
                   0.012169, 0.010621, 0.009583, 0.007563, 0.007324, 0.003711, 0.003462, 0.003390, 0.003144,
                   0.003144, 0.003092, 0.000909, 0.000000, 0.000000]
}

# Creating DataFrames
df_2022_q4_decline = pd.DataFrame(data_2022_q4_decline)
df_2022_q4_rise = pd.DataFrame(data_2022_q4_rise)

df_2022_q4_decline, df_2022_q4_rise

# Merging the two dataframes
# Since the 'Feature' columns are the same, we can merge on this column
merged_df = pd.merge(df_2022_q4_decline, df_2022_q4_rise, on='Feature', suffixes=('_decline', '_rise'))

merged_df.head()


,Feature,Importance_decline,Importance_rise
0,투기지역,0.412253,0.003144
1,중학교_학업성취도,0.119165,0.056460
2,투기과열지구,0.084196,0.000000
3,건축년도,0.058413,0.254643
4,직주_근접도,0.049739,0.003092


In [ ]:
correlations = df_subset_2022_1Q.corr()
unit_price_correlations = correlations['단위면적당거래금액'].sort_values(ascending = False)
print(unit_price_correlations)

단위면적당거래금액       1.000000
건축년도            0.502541
세대당_주차대수        0.467295
조정대상지구          0.443347
세대수             0.423157
중학교_학업성취도       0.314212
마트_유무           0.202957
건설사             0.201583
전용면적구간          0.196764
스타벅스_매장수        0.193632
간접역세권_노선수       0.184926
백화점쇼핑몰_유무       0.179518
고속철_직접역세권_여부    0.078602
용적률             0.067729
직접역세권_노선수       0.062865
직주_근접도          0.059111
투기지역            0.014476
임대세대비율         -0.013404
건폐율            -0.014336
중학교_최단거리       -0.081148
초등학교_최단거리구간    -0.150606
현관구조           -0.181862
학원가_근접도              NaN
투기과열지구               NaN
Name: 단위면적당거래금액, dtype: float64


In [ ]:
!pip install boruta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 662.0 kB/s eta 0:00:00


In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

# Define the features and target variable
X = df_subset_2019_3Q.drop('단위면적당거래금액', axis=1)
y = df_subset_2019_3Q['단위면적당거래금액']

# Initialize RandomForestRegressor
forest = RandomForestRegressor(n_jobs=-1, max_depth=5, random_state=42)

# Initialize Boruta
boruta = BorutaPy(forest, n_estimators='auto', random_state=1)

# Fit Boruta
boruta.fit(X.values, y.values)


BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=105,
                                         n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x7B9B79F16540),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7B9B79F16540)

In [ ]:
# 중요한 특성들을 출력
print("Selected Features: ", X.columns[boruta.support_].tolist())

# 결정이 확실하지 않은 특성들을 출력
print("Tentative Features: ", X.columns[boruta.support_weak_].tolist())


Selected Features:  ['세대수', '용적률', '건폐율', '세대당_주차대수', '직접역세권_노선수', '간접역세권_노선수', '고속철_직접역세권_여부', '중학교_학업성취도', '중학교_최단거리', '직주_근접도', '학원가_근접도', '건축년도', '백화점쇼핑몰_유무', '마트_유무']
Tentative Features:  []
